In [1]:
import pandas as pd

In [2]:
from pymongo import MongoClient
from newsapi import NewsApiClient
from newspaper import Article
from datetime import datetime
import pytz
from newspaper import Article, ArticleException
from requests.exceptions import HTTPError
from langchain import OpenAI, ConversationChain, LLMChain, PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from pymongo import MongoClient
from urllib.parse import urlparse
from dotenv import load_dotenv
import os

In [3]:
api_key = os.getenv("OPENAI_API_KEY")

Codes below is how we made ChatGPT predict news relevancy:

In [4]:
def relevance_GPT(article_insert):
    print('RELEVANCE GPT FUNCTION WORKING')
    try:
        #Defining Function + ChatGPT
        #Langchain implementation
        template = """ You are a bot that will be given an article and to deem if it is relevant to technology. 
        Please answer with Relevant or Irrelevant 

        Technology also includes the following:
        
        AI includes Discriminative AI, Machine Learning, Generative AI

        Quantum Computing includes Quantum Internet, Quantum Communications, Quatum Computing

        Green Computing includes Green Serverless Computing, Green Edge Applications, Green Data Streaming

        Robotics

        Trust Technologies includes Privacy Enhancing Technologies, Regulation Technologies, Al Governance Technologies

        Anti-disinformation technologies includes Content Provenance Technologies, Anti-misinformation technologies, Detection of Generated Al content

        Communications Technologies includes 5G, Networks, Seamless

        In terms of the article information, classify them as Relevant or Irrelevant. 

        Make sure that there are no spacing before the first word.


        Human: {article}
        Assistant:"""

        prompt = PromptTemplate(
            input_variables=["article"], 
            template=template
        )

        chatgpt_chain = LLMChain(
            llm = OpenAI(openai_api_key=api_key,model="gpt-3.5-turbo-instruct", temperature=0), 
            prompt=prompt, 
            verbose=False,  # Set verbose to False to suppress output
            memory=ConversationBufferMemory(memory_key="history", input_key="article")
        )
        #Let ChatGPT to Categorise
        output = chatgpt_chain.predict(article=article_insert) 

        #To make sure that there are no spacing which ChatGPT outputs " Category_Name" -> "Category_Name"
        output = output.strip()

        return output

    except Exception as e:
        print(f"An error occurred: {e}")
        return "Irrelevant"  # Return 'Irrelevant'' category in case of error

In [5]:
# Read the CSV file into a DataFrame
df = pd.read_csv('news relevancy - api_data.csv')
#Def bart apply to be used for pandas apply function
def gpt_rev(x):
    result = relevance_GPT(x)
    return result


df['cat'] = df['Article'].apply(gpt_rev)

column_to_export = df['cat']

# Export the column to a CSV file
column_to_export.to_csv('gpt_column.csv', index=False)

print("Column exported successfully.")

RELEVANCE GPT FUNCTION WORKING


/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


RELEVANCE GPT FUNCTION WORKING
RELEVANCE GPT FUNCTION WORKING
RELEVANCE GPT FUNCTION WORKING
RELEVANCE GPT FUNCTION WORKING
RELEVANCE GPT FUNCTION WORKING


KeyboardInterrupt: 

Chat-GPT Accuracy in Predicting Relevancy

In [62]:
def acc_prediction(predicted_csv):
    # Read the CSV file into a DataFrame
    df = pd.read_csv('news relevancy - api_data.csv')
    print(predicted_csv)
    df_2 = pd.read_csv(predicted_csv)
    #df_2 = pd.read_csv(predicted_csv)

    # List of categories to label as 1
    label_1_categories = ["Business","General","Finance"]

    # Apply a lambda function to assign labels
    df['bart_col'] = df_2.iloc[:, 0].apply(lambda x: 0 if x in label_1_categories else 1)


    # Replace "Yes" and "No" with 1s and 0s respectively
    df.replace({"Relevant": 1, "Irrelevant": 0}, inplace=True)

    # Separate predictions and ground truth
    predictions = df['bart_col']  # Assuming predictions are in the first row
    ground_truth = df['groundTruth']  # Assuming ground truth is in the second row


    #print(predictions)

    # Compare predictions and ground truth
    correct_predictions = (predictions == ground_truth).sum()
    total_predictions = len(predictions)

    # Print results
    print("Predictions:", predictions.values)
    print("Ground Truth:", ground_truth.values)
    print("Number of correct predictions:", correct_predictions)
    print("Total predictions made:", total_predictions)
    print("Accuracy:", correct_predictions / total_predictions)

In [56]:
#Chat GPT Prediction
acc_prediction('gpt_column.csv')

gpt_column.csv
Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
Ground Truth: [1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 1
 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1
 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1
 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 0 0 0
 0 0]
Number of correct predictions: 95
Total predictions made: 150
Accuracy: 0.6333333333333333


In [126]:
#Bart Prediction
acc_prediction('bart_column_updated.csv')

bart_column_updated.csv
Predictions: [1 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 1
 0 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1
 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 0
 0 0]
Ground Truth: [1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 1
 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1
 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1
 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 0 0 0
 0 0]
Number of correct predictions: 115
Total predictions made: 150
Accuracy: 0.7666666666666667


In [68]:
#deberta
acc_prediction('deberta_column.csv')

deberta_column.csv
Predictions: [1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1
 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 1 1 0 0 1
 1 0 1 0 1 0 0 1 1 1 1 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0 0 0 0
 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 1 1 1 0 0 0 0 0 0
 0 0]
Ground Truth: [1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 1
 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1
 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1
 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 0 0 0
 0 0]
Number of correct predictions: 116
Total predictions made: 150
Accuracy: 0.7733333333333333


In [131]:
#voting
df = pd.read_csv('news relevancy - api_data.csv')
df_2 = pd.read_csv('voting_column.csv')

# Replace "Yes" and "No" with 1s and 0s respectively
df.replace({"Relevant": 1, "Irrelevant": 0}, inplace=True)
df_2.replace({"Relevant": 1, "Irrelevant": 0}, inplace=True)

# Separate predictions and ground truth
predictions = df_2.iloc[:,0]  
ground_truth = df['groundTruth']  

# Compare predictions and ground truth
correct_predictions = (predictions == ground_truth).sum()
total_predictions = len(predictions)

# Print results
print("Predictions:", predictions.values)
print("Ground Truth:", ground_truth.values)
print("Number of correct predictions:", correct_predictions)
print("Total predictions made:", total_predictions)
print("Accuracy:", correct_predictions / total_predictions)

Predictions: [1 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1
 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1
 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 0 0 0 0 0
 0 0]
Ground Truth: [1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 1 1
 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1
 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 1
 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 0 0 0
 0 0]
Number of correct predictions: 122
Total predictions made: 150
Accuracy: 0.8133333333333334
